### Pipeline optimization 
* Objective:
 * Build a stack of numpy arrays depicting the cost distance away from biogas sources
 
* Requirements - Packages
 * NumPy, Pandas, Scikit-Image

* Requirements - Data:
 * CSV of biogas sources, including coordinates and biogas production potential (MMBtu)
 * Cost surface for building pipelines
 
* Output: 
 * Stacked NumPy array
 
 https://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#create-least-cost-path

In [1]:
#Import packages
import numpy as np
import pandas as pd
from skimage import graph
from osgeo import gdal
from matplotlib import pyplot as plt

ImportError: DLL load failed: The specified module could not be found.

In [ ]:
#Read in biogase sources (example: Duplin Co)
dfBG =  pd.read_excel('../data/DuplinCountySwineFarmEconomics.xlsx',
                         sheet_name='Duplin County Swine Farm Master').iloc[:,[11,12,16]]

In [ ]:
#Read in cost surface
ds = gdal.Open('../data/MIT Cost Surface Duplin Co/MIT Cost Surface Duplin.tif')
#Get raster attributes
llx, x_size, x_angle, lly, y_angle, y_size = ds.GetGeoTransform()
arrCost = np.array(ds.GetRasterBand(1).ReadAsArray())
print("Lower left coordinate = ({},{})".format(llx,lly))
print("Pixel size is {}(x), {}(y)".format(x_size,y_size))

In [ ]:
#Create the MCP object from the cost surface
print("Creating the graph from the cost raster")
lc_graph = graph.MCP_Geometric(arrCost,sampling=(x_size,y_size))

In [ ]:
y,x = dfBG.iloc[0,[0,1]].values
print(x,y)

In [ ]:
#Get array index from lat long
xOffset = int((x - llx)/x_size)
yOffset = int((y - lly)/y_size)
xOffset,yOffset

In [ ]:
#Get value at the array index
arrCost_mod = arrCost.copy()
arrCost_mod[xOffset,yOffset] = 0

In [ ]:
list(zip([xOffset],[yOffset]))

In [ ]:
#Compute cost distances away
lcd = lg.find_costs(starts=([(xOffset, yOffset)]))[0]

In [ ]:
lcd

In [ ]:
df get_cdArray(costGraph,x,y,x_size,y_size):
    '''Returns a cost distance array from the origin coordinate provided.
    '''
    #Create the MCP object from the cost array
    lg = graph.MCP_Geometric(arrCost_mod,sampling=(x_size,y_size))
    #


In [ ]:
def raster2array(rasterfn):
    raster = gdal.Open(rasterfn)
    band = raster.GetRasterBand(1)
    array = band.ReadAsArray()
    return array

def coord2pixelOffset(rasterfn,x,y):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    xOffset = int((x - originX)/pixelWidth)
    yOffset = int((y - originY)/pixelHeight)
    return xOffset,yOffset

def createPath(CostSurfacefn,costSurfaceArray,startCoord,stopCoord):

    # coordinates to array index
    startCoordX = startCoord[0]
    startCoordY = startCoord[1]
    startIndexX,startIndexY = coord2pixelOffset(CostSurfacefn,startCoordX,startCoordY)

    stopCoordX = stopCoord[0]
    stopCoordY = stopCoord[1]
    stopIndexX,stopIndexY = coord2pixelOffset(CostSurfacefn,stopCoordX,stopCoordY)

    # create path
    indices, weight = route_through_array(costSurfaceArray, (startIndexY,startIndexX), (stopIndexY,stopIndexX),geometric=True,fully_connected=True)
    indices = np.array(indices).T
    path = np.zeros_like(costSurfaceArray)
    path[indices[0], indices[1]] = 1
    return path

def array2raster(newRasterfn,rasterfn,array):
    raster = gdal.Open(rasterfn)
    geotransform = raster.GetGeoTransform()
    originX = geotransform[0]
    originY = geotransform[3]
    pixelWidth = geotransform[1]
    pixelHeight = geotransform[5]
    cols = array.shape[1]
    rows = array.shape[0]

    driver = gdal.GetDriverByName('GTiff')
    outRaster = driver.Create(newRasterfn, cols, rows, 1, gdal.GDT_Byte)
    outRaster.SetGeoTransform((originX, pixelWidth, 0, originY, 0, pixelHeight))
    outband = outRaster.GetRasterBand(1)
    outband.WriteArray(array)
    outRasterSRS = osr.SpatialReference()
    outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
    outRaster.SetProjection(outRasterSRS.ExportToWkt())
    outband.FlushCache()

def main(CostSurfacefn,outputPathfn,startCoord,stopCoord):

    costSurfaceArray = raster2array(CostSurfacefn) # creates array from cost surface raster

    pathArray = createPath(CostSurfacefn,costSurfaceArray,startCoord,stopCoord) # creates path array

    array2raster(outputPathfn,CostSurfacefn,pathArray) # converts path array to raster
